# U-Net for Snow Segmentation in Aerial Images

In [ ]:
# Install required libraries
pip install tensorflow scikit-image
    

### Data Preparation

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.io import imread

def load_data(image_paths, mask_paths):
    images = [imread(img_path) for img_path in image_paths]
    masks = [imread(mask_path) for mask_path in mask_paths]
    return np.array(images), np.array(masks)
    

In [11]:
#gpu stuff
print("Num GPUs Available: ", tf.config.list_physical_devices())



Num GPUs Available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


### Define U-Net Model

In [2]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model

def unet(input_size=(None, None, 3)):
    inputs = Input(input_size)
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    
    # Decoder
    up6 = Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(conv5)
    up6 = concatenate([up6, conv4])
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)
    
    up7 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    up7 = concatenate([up7, conv3])
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)
    
    up8 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
    up8 = concatenate([up8, conv2])
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)
    
    up9 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv8)
    up9 = concatenate([up9, conv1])
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

model = unet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    

### Data Augmentation and Training

In [8]:
#get image and mask paths
#convert to dataframe
import pandas as pd
import os
df = pd.read_csv('data/2019-20/2019-20.csv')

image_paths = np.array(df['snow_path'].values)
image_paths = [os.path.join('data', path) for path in image_paths]
image_paths

['data\\./2019-20\\DeerCreekTrail_2019_05_22_snow.tif',
 'data\\./2019-20\\DeerCreekTrail_2019_05_30_snow.tif',
 'data\\./2019-20\\DeerCreekTrail_2019_06_07_snow.tif',
 'data\\./2019-20\\DeerCreekTrail_2019_06_13_snow.tif',
 'data\\./2019-20\\DeerCreekTrail_2019_06_19_snow.tif',
 'data\\./2019-20\\DeerCreekTrail_2019_07_02_snow.tif',
 'data\\./2019-20\\DeerCreekTrail_2020_05_05_snow.tif',
 'data\\./2019-20\\DeerCreekTrail_2020_05_12_snow.tif',
 'data\\./2019-20\\DeerCreekTrail_2020_05_23_snow.tif',
 'data\\./2019-20\\DeerCreekTrail_2020_05_28_snow.tif',
 'data\\./2019-20\\DeerCreekTrail_2020_06_04_snow.tif',
 'data\\./2019-20\\DeerCreekTrail_2020_06_13_snow.tif',
 'data\\./2019-20\\EastRiverTrail_2019_05_03_snow.tif',
 'data\\./2019-20\\EastRiverTrail_2019_05_08_snow.tif',
 'data\\./2019-20\\EastRiverTrail_2019_05_12_snow.tif',
 'data\\./2019-20\\EastRiverTrail_2019_05_19_snow.tif',
 'data\\./2019-20\\EastRiverTrail_2019_05_24_snow.tif',
 'data\\./2019-20\\EastRiverTrail_2019_05_30_sno

In [ ]:


# Assuming your images and masks are numpy arrays
image_paths = [...]  # list of paths to your .tif images
mask_paths = [...]   # list of paths to your corresponding masks

images, masks = load_data(image_paths, mask_paths)
masks = masks[..., np.newaxis]  # add a channel dimension if necessary

data_gen_args = dict(rotation_range=10.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     shear_range=0.2,
                     zoom_range=0.2,
                     horizontal_flip=True,
                     fill_mode='nearest')

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

image_datagen.fit(images, augment=True, seed=1)
mask_datagen.fit(masks, augment=True, seed=1)

image_generator = image_datagen.flow(images, batch_size=32, seed=1)
mask_generator = mask_datagen.flow(masks, batch_size=32, seed=1)

train_generator = zip(image_generator, mask_generator)

model.fit(train_generator, steps_per_epoch=len(images) // 32, epochs=50)
    